In [ ]:
# MNIST training using CNN

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
nb_classes = 10
x_train = tf.reshape(x_train, [-1, 28, 28, 1])
x_test = tf.reshape(x_test, [-1, 28, 28, 1])
y_train = tf.one_hot(y_train, depth=nb_classes)
y_test = tf.one_hot(y_test, depth=nb_classes)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))

training_epochs = 15
batch_size = 100

dataset = train_data.batch(batch_size)

# for features, label in dataset.take(1):
    # print(features, label)

initializer = tf.keras.initializers.GlorotUniform()

W1 = tf.Variable(initializer([3, 3, 1, 32]))

W2 = tf.Variable(initializer([3, 3, 32, 64]))

W3 = tf.Variable(initializer([7*7*64, nb_classes]))
b = tf.Variable(tf.random.normal([nb_classes]))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

def compute_cost(X, Y):
    X = tf.cast(X, dtype=tf.float32) / 255.0
    L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding="SAME")
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    L2 = tf.reshape(L2, [-1, 7*7*64])
    hypothesis = tf.matmul(L2, W3) + b
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    return hypothesis, cost

def train_step(X, Y):
    with tf.GradientTape() as tape:
        hypothesis, cost = compute_cost(X, Y)
    gradients  = tape.gradient(cost, [W1, W2, W3, b])
    optimizer.apply_gradients(zip(gradients, [W1, W2, W3, b]))
    return hypothesis, cost

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(train_data) / batch_size

    for batch in dataset:
        x_batch, y_batch = batch
        hypothesis, cost_val = train_step(x_batch, y_batch)
        avg_cost += cost_val / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost", "{:.9f}".format(avg_cost))

hypothesis, cost_val = compute_cost(x_test, y_test,)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y_test, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("Accuracy: ", accuracy)

/Users/heayounchoi/anaconda3/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: 0001 cost 0.250495911
Epoch: 0002 cost 0.068430156
Epoch: 0003 cost 0.048835058
Epoch: 0004 cost 0.039090190
Epoch: 0005 cost 0.032207254
Epoch: 0006 cost 0.026936965
Epoch: 0007 cost 0.022801897
Epoch: 0008 cost 0.018768203
Epoch: 0009 cost 0.015232667
Epoch: 0010 cost 0.012859721
Epoch: 0011 cost 0.010880950
Epoch: 0012 cost 0.010811281
Epoch: 0013 cost 0.008747036
Epoch: 0014 cost 0.006726051
Epoch: 0015 cost 0.006639170
Accuracy:  tf.Tensor(0.9875, shape=(), dtype=float32)


In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
nb_classes = 10
x_train = tf.reshape(x_train, [-1, 28, 28, 1])
x_test = tf.reshape(x_test, [-1, 28, 28, 1])
y_train = tf.one_hot(y_train, depth=nb_classes)
y_test = tf.one_hot(y_test, depth=nb_classes)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))

training_epochs = 15
batch_size = 100

dataset = train_data.batch(batch_size)

# for features, label in dataset.take(1):
    # print(features, label)

initializer = tf.keras.initializers.GlorotUniform()

W1 = tf.Variable(initializer([3, 3, 1, 32]))

W2 = tf.Variable(initializer([3, 3, 32, 64]))

W3 = tf.Variable(initializer([3, 3, 64, 128]))

W4 = tf.Variable(initializer([128*4*4, 625]))
b4 = tf.Variable(tf.random.normal(([625])))

W5 = tf.Variable(initializer([625, nb_classes]))
b5 = tf.Variable(tf.random.normal([nb_classes]))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

def compute_cost(X, Y, dropout_rate):
    X = tf.cast(X, dtype=tf.float32) / 255.0
    
    L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding="SAME")
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    L1 = tf.nn.dropout(L1, rate = dropout_rate)
    
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    L2 = tf.nn.dropout(L2, rate = dropout_rate)
    
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
    L3 = tf.nn.dropout(L3, rate = dropout_rate)
    L3 = tf.reshape(L3, [-1, 128 * 4 * 4])
    
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 - tf.nn.dropout(L4, rate = dropout_rate)
    
    hypothesis = tf.matmul(L4, W5) + b5
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    return hypothesis, cost

def train_step(X, Y, dropout_rate):
    with tf.GradientTape() as tape:
        hypothesis, cost = compute_cost(X, Y, dropout_rate)
    gradients  = tape.gradient(cost, [W1, W2, W3, W4, W5, b4, b5])
    optimizer.apply_gradients(zip(gradients, [W1, W2, W3, W4, W5, b4, b5]))
    return hypothesis, cost

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(train_data) / batch_size

    for batch in dataset:
        x_batch, y_batch = batch
        hypothesis, cost_val = train_step(x_batch, y_batch, 0.3)
        avg_cost += cost_val / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost", "{:.9f}".format(avg_cost))

hypothesis, cost_val = compute_cost(x_test, y_test, 0)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y_test, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("Accuracy: ", accuracy)

Epoch: 0001 cost 0.258709311
Epoch: 0002 cost 0.074086808
Epoch: 0003 cost 0.053217709
Epoch: 0004 cost 0.044324409
Epoch: 0005 cost 0.040001828
Epoch: 0006 cost 0.034401406
Epoch: 0007 cost 0.032023497
Epoch: 0008 cost 0.027105700
Epoch: 0009 cost 0.026817592
Epoch: 0010 cost 0.026598470
Epoch: 0011 cost 0.021547828
Epoch: 0012 cost 0.020646309
Epoch: 0013 cost 0.020462535
Epoch: 0014 cost 0.019768706
Epoch: 0015 cost 0.018117338
Accuracy:  tf.Tensor(0.9932, shape=(), dtype=float32)
